In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os
from torch.utils import data
from wavenet import Wavenet
from transformData import x_mu_law_encode,y_mu_law_encode,mu_law_decode,onehot,cateToSignal
from readDataset import Dataset

In [2]:
sampleSize=32000#the length of the sample size
quantization_channels=256
sample_rate=16000
dilations=[2**i for i in range(9)]*3  #idea from wavenet, have more receptive field
residualDim=128 #
skipDim=512
shapeoftest = 190500
filterSize=3
resumefile='ccmix' # name of checkpoint
lossname='ccmixloss.txt' # name of loss file
continueTrain=False # whether use checkpoint
pad = np.sum(dilations) # padding for dilate convolutional layers
lossrecord=[]  #list for record loss
#pad=0

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use specific GPU

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} 

In [5]:
params = {'batch_size': 1,'shuffle': True,'num_workers': 2}

training_set = Dataset(np.arange(0,3),np.arange(0,3),'ccmixter/x/','ccmixter/y/')
validation_set=Dataset(np.arange(3,4),np.arange(3,4),'ccmixter/x/','ccmixter/y/')
loadtr = data.DataLoader(training_set, **params)
loadval = data.DataLoader(validation_set, **params)

In [6]:
model = Wavenet(pad,skipDim,quantization_channels,residualDim,dilations).cuda()
criterion = nn.CrossEntropyLoss()
#in wavenet paper, they said crossentropyloss is far better than MSELoss
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
#use adam to train
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

In [7]:
if continueTrain:# if continueTrain, the program will find the checkpoints
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

In [ ]:
def val(): # validation set
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        for iloader,(xtrain,ytrain) in enumerate(loadval):
            idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,1000)
            np.random.shuffle(idx)
            data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
            target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
            val_loss = criterion(output, target).item()
            print(correct,'accurate')
            print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

            listofpred = []
            for ind in range(pad,xtrain.shape[-1]-pad-sampleSize,sampleSize):
                output = model(xtrain[:, :, ind - pad:ind + sampleSize + pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/noteval.wav', ans, sample_rate)
            break
    
def train(epoch): #training set
    model.train()
    for iloader,(xtrain,ytrain) in enumerate(loadtr):
        idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,16000)
        np.random.shuffle(idx)#random the starting points
        for i, ind in enumerate(idx):
            start_time = time.time()
            data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
            output = model(data)
            loss = criterion(output, target)
            lossrecord.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('Train Epoch: {} iloader:{} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                    epoch, iloader,i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
            if i % 100 == 0:
                with open("./lossRecord/"+lossname, "w") as f:
                    for s in lossrecord:
                        f.write(str(s) +"\n")
                print('write finish')
                state={'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()}
                #torch.save(state, './model/'+resumefile)
        val()

In [ ]:
for epoch in range(100000):
    train(epoch)

Train Epoch: 0 iloader:0 [0/167 (0%)] Loss:5.552440: , (2.146 sec/step)
write finish
Train Epoch: 0 iloader:0 [1/167 (1%)] Loss:5.479185: , (1.265 sec/step)
Train Epoch: 0 iloader:0 [2/167 (1%)] Loss:5.774625: , (1.264 sec/step)
Train Epoch: 0 iloader:0 [3/167 (2%)] Loss:5.461481: , (1.264 sec/step)
Train Epoch: 0 iloader:0 [4/167 (2%)] Loss:5.368350: , (1.265 sec/step)
Train Epoch: 0 iloader:0 [5/167 (3%)] Loss:5.396144: , (1.264 sec/step)
Train Epoch: 0 iloader:0 [6/167 (4%)] Loss:5.752612: , (1.266 sec/step)
Train Epoch: 0 iloader:0 [7/167 (4%)] Loss:5.005978: , (1.262 sec/step)
Train Epoch: 0 iloader:0 [8/167 (5%)] Loss:5.367545: , (1.923 sec/step)
Train Epoch: 0 iloader:0 [9/167 (5%)] Loss:5.534677: , (2.522 sec/step)
Train Epoch: 0 iloader:0 [10/167 (6%)] Loss:5.449419: , (2.536 sec/step)
Train Epoch: 0 iloader:0 [11/167 (7%)] Loss:3.386994: , (2.555 sec/step)
Train Epoch: 0 iloader:0 [12/167 (7%)] Loss:2.227281: , (2.562 sec/step)
Train Epoch: 0 iloader:0 [13/167 (8%)] Loss:0.88

Train Epoch: 0 iloader:0 [111/167 (66%)] Loss:5.962584: , (2.537 sec/step)
Train Epoch: 0 iloader:0 [112/167 (67%)] Loss:5.803764: , (2.544 sec/step)
Train Epoch: 0 iloader:0 [113/167 (68%)] Loss:5.971984: , (2.507 sec/step)
Train Epoch: 0 iloader:0 [114/167 (68%)] Loss:5.476254: , (2.448 sec/step)
Train Epoch: 0 iloader:0 [115/167 (69%)] Loss:5.278788: , (2.550 sec/step)
Train Epoch: 0 iloader:0 [116/167 (69%)] Loss:5.166112: , (2.539 sec/step)
Train Epoch: 0 iloader:0 [117/167 (70%)] Loss:4.870385: , (2.538 sec/step)
Train Epoch: 0 iloader:0 [118/167 (71%)] Loss:4.958334: , (2.533 sec/step)
Train Epoch: 0 iloader:0 [119/167 (71%)] Loss:5.156715: , (2.530 sec/step)
Train Epoch: 0 iloader:0 [120/167 (72%)] Loss:5.007924: , (2.521 sec/step)
Train Epoch: 0 iloader:0 [121/167 (72%)] Loss:5.079679: , (2.423 sec/step)
Train Epoch: 0 iloader:0 [122/167 (73%)] Loss:5.102778: , (2.546 sec/step)
Train Epoch: 0 iloader:0 [123/167 (74%)] Loss:5.097556: , (2.533 sec/step)
Train Epoch: 0 iloader:0 

Train Epoch: 0 iloader:1 [54/198 (27%)] Loss:4.633418: , (2.656 sec/step)
Train Epoch: 0 iloader:1 [55/198 (28%)] Loss:4.613002: , (2.645 sec/step)
Train Epoch: 0 iloader:1 [56/198 (28%)] Loss:4.830212: , (2.671 sec/step)
Train Epoch: 0 iloader:1 [57/198 (29%)] Loss:4.506245: , (2.683 sec/step)
Train Epoch: 0 iloader:1 [58/198 (29%)] Loss:4.983560: , (2.667 sec/step)
Train Epoch: 0 iloader:1 [59/198 (30%)] Loss:4.921187: , (2.671 sec/step)
Train Epoch: 0 iloader:1 [60/198 (30%)] Loss:4.397872: , (2.657 sec/step)
Train Epoch: 0 iloader:1 [61/198 (31%)] Loss:4.659589: , (2.647 sec/step)
Train Epoch: 0 iloader:1 [62/198 (31%)] Loss:4.787715: , (2.650 sec/step)
Train Epoch: 0 iloader:1 [63/198 (32%)] Loss:4.540752: , (2.685 sec/step)
Train Epoch: 0 iloader:1 [64/198 (32%)] Loss:4.901651: , (2.696 sec/step)
Train Epoch: 0 iloader:1 [65/198 (33%)] Loss:4.877369: , (2.066 sec/step)
Train Epoch: 0 iloader:1 [66/198 (33%)] Loss:4.863883: , (1.263 sec/step)
Train Epoch: 0 iloader:1 [67/198 (34%)

Train Epoch: 0 iloader:1 [164/198 (83%)] Loss:4.801022: , (2.534 sec/step)
Train Epoch: 0 iloader:1 [165/198 (83%)] Loss:4.472404: , (2.544 sec/step)
Train Epoch: 0 iloader:1 [166/198 (84%)] Loss:4.439231: , (2.537 sec/step)
Train Epoch: 0 iloader:1 [167/198 (84%)] Loss:4.392233: , (2.403 sec/step)
Train Epoch: 0 iloader:1 [168/198 (85%)] Loss:4.744202: , (2.537 sec/step)
Train Epoch: 0 iloader:1 [169/198 (85%)] Loss:4.296154: , (2.526 sec/step)
Train Epoch: 0 iloader:1 [170/198 (86%)] Loss:4.362574: , (2.489 sec/step)
Train Epoch: 0 iloader:1 [171/198 (86%)] Loss:4.145913: , (2.523 sec/step)
Train Epoch: 0 iloader:1 [172/198 (87%)] Loss:4.204517: , (2.524 sec/step)
Train Epoch: 0 iloader:1 [173/198 (87%)] Loss:4.935669: , (2.526 sec/step)
Train Epoch: 0 iloader:1 [174/198 (88%)] Loss:4.762069: , (2.535 sec/step)
Train Epoch: 0 iloader:1 [175/198 (88%)] Loss:4.406997: , (2.412 sec/step)
Train Epoch: 0 iloader:1 [176/198 (89%)] Loss:4.923156: , (2.531 sec/step)
Train Epoch: 0 iloader:1 

Train Epoch: 0 iloader:2 [76/213 (36%)] Loss:4.626308: , (2.649 sec/step)
Train Epoch: 0 iloader:2 [77/213 (36%)] Loss:4.511706: , (2.635 sec/step)
Train Epoch: 0 iloader:2 [78/213 (37%)] Loss:4.891651: , (2.687 sec/step)
Train Epoch: 0 iloader:2 [79/213 (37%)] Loss:4.660375: , (2.691 sec/step)
Train Epoch: 0 iloader:2 [80/213 (38%)] Loss:4.356632: , (2.690 sec/step)
Train Epoch: 0 iloader:2 [81/213 (38%)] Loss:5.186383: , (2.702 sec/step)
Train Epoch: 0 iloader:2 [82/213 (38%)] Loss:4.716727: , (2.657 sec/step)
Train Epoch: 0 iloader:2 [83/213 (39%)] Loss:4.578950: , (2.665 sec/step)
Train Epoch: 0 iloader:2 [84/213 (39%)] Loss:3.060394: , (2.683 sec/step)
Train Epoch: 0 iloader:2 [85/213 (40%)] Loss:4.853514: , (2.687 sec/step)
Train Epoch: 0 iloader:2 [86/213 (40%)] Loss:2.426726: , (2.675 sec/step)
Train Epoch: 0 iloader:2 [87/213 (41%)] Loss:4.424551: , (2.651 sec/step)
Train Epoch: 0 iloader:2 [88/213 (41%)] Loss:5.099480: , (2.636 sec/step)
Train Epoch: 0 iloader:2 [89/213 (42%)

...

Train Epoch: 66 iloader:2 [75/213 (35%)] Loss:3.794033: , (1.276 sec/step)
Train Epoch: 66 iloader:2 [76/213 (36%)] Loss:3.510058: , (1.277 sec/step)
Train Epoch: 66 iloader:2 [77/213 (36%)] Loss:2.612074: , (1.279 sec/step)
Train Epoch: 66 iloader:2 [78/213 (37%)] Loss:3.247715: , (1.276 sec/step)
Train Epoch: 66 iloader:2 [79/213 (37%)] Loss:3.356586: , (1.277 sec/step)
Train Epoch: 66 iloader:2 [80/213 (38%)] Loss:3.595711: , (1.281 sec/step)
Train Epoch: 66 iloader:2 [81/213 (38%)] Loss:3.149471: , (1.278 sec/step)
Train Epoch: 66 iloader:2 [82/213 (38%)] Loss:3.492179: , (1.280 sec/step)
Train Epoch: 66 iloader:2 [83/213 (39%)] Loss:3.454642: , (1.276 sec/step)
Train Epoch: 66 iloader:2 [84/213 (39%)] Loss:3.412600: , (1.277 sec/step)
Train Epoch: 66 iloader:2 [85/213 (40%)] Loss:3.555840: , (1.276 sec/step)
Train Epoch: 66 iloader:2 [86/213 (40%)] Loss:1.645102: , (1.276 sec/step)
Train Epoch: 66 iloader:2 [87/213 (41%)] Loss:1.218287: , (1.279 sec/step)
Train Epoch: 66 iloader:2

Train Epoch: 66 iloader:2 [183/213 (86%)] Loss:1.668917: , (1.282 sec/step)
Train Epoch: 66 iloader:2 [184/213 (86%)] Loss:3.403532: , (1.282 sec/step)
Train Epoch: 66 iloader:2 [185/213 (87%)] Loss:3.276789: , (1.282 sec/step)
Train Epoch: 66 iloader:2 [186/213 (87%)] Loss:3.428282: , (1.292 sec/step)
Train Epoch: 66 iloader:2 [187/213 (88%)] Loss:3.517057: , (1.282 sec/step)
Train Epoch: 66 iloader:2 [188/213 (88%)] Loss:3.188732: , (1.286 sec/step)
Train Epoch: 66 iloader:2 [189/213 (89%)] Loss:3.193902: , (1.284 sec/step)
Train Epoch: 66 iloader:2 [190/213 (89%)] Loss:0.698686: , (1.280 sec/step)
Train Epoch: 66 iloader:2 [191/213 (90%)] Loss:3.484904: , (1.281 sec/step)
Train Epoch: 66 iloader:2 [192/213 (90%)] Loss:3.733585: , (1.286 sec/step)
Train Epoch: 66 iloader:2 [193/213 (91%)] Loss:3.305798: , (1.290 sec/step)
Train Epoch: 66 iloader:2 [194/213 (91%)] Loss:3.473242: , (1.282 sec/step)
Train Epoch: 66 iloader:2 [195/213 (92%)] Loss:3.168164: , (1.282 sec/step)
Train Epoch:

Train Epoch: 67 iloader:0 [79/198 (40%)] Loss:3.582503: , (1.282 sec/step)
Train Epoch: 67 iloader:0 [80/198 (40%)] Loss:3.609657: , (1.286 sec/step)
Train Epoch: 67 iloader:0 [81/198 (41%)] Loss:3.780687: , (1.287 sec/step)
Train Epoch: 67 iloader:0 [82/198 (41%)] Loss:3.782370: , (1.282 sec/step)
Train Epoch: 67 iloader:0 [83/198 (42%)] Loss:3.522707: , (1.289 sec/step)
Train Epoch: 67 iloader:0 [84/198 (42%)] Loss:3.815433: , (1.285 sec/step)
Train Epoch: 67 iloader:0 [85/198 (43%)] Loss:3.994496: , (1.284 sec/step)
Train Epoch: 67 iloader:0 [86/198 (43%)] Loss:3.671420: , (1.285 sec/step)
Train Epoch: 67 iloader:0 [87/198 (44%)] Loss:3.618627: , (1.284 sec/step)
Train Epoch: 67 iloader:0 [88/198 (44%)] Loss:3.366207: , (1.287 sec/step)
Train Epoch: 67 iloader:0 [89/198 (45%)] Loss:3.932082: , (1.288 sec/step)
Train Epoch: 67 iloader:0 [90/198 (45%)] Loss:3.562516: , (1.289 sec/step)
Train Epoch: 67 iloader:0 [91/198 (46%)] Loss:3.495028: , (1.285 sec/step)
Train Epoch: 67 iloader:0

Train Epoch: 67 iloader:0 [187/198 (94%)] Loss:3.636496: , (1.273 sec/step)
Train Epoch: 67 iloader:0 [188/198 (95%)] Loss:3.523704: , (1.275 sec/step)
Train Epoch: 67 iloader:0 [189/198 (95%)] Loss:3.926245: , (1.274 sec/step)
Train Epoch: 67 iloader:0 [190/198 (96%)] Loss:3.446277: , (1.275 sec/step)
Train Epoch: 67 iloader:0 [191/198 (96%)] Loss:3.635236: , (1.274 sec/step)
Train Epoch: 67 iloader:0 [192/198 (97%)] Loss:3.884879: , (1.277 sec/step)
Train Epoch: 67 iloader:0 [193/198 (97%)] Loss:3.779434: , (1.275 sec/step)
Train Epoch: 67 iloader:0 [194/198 (98%)] Loss:3.607658: , (1.274 sec/step)
Train Epoch: 67 iloader:0 [195/198 (98%)] Loss:3.820345: , (1.278 sec/step)
Train Epoch: 67 iloader:0 [196/198 (99%)] Loss:3.845307: , (1.278 sec/step)
Train Epoch: 67 iloader:0 [197/198 (99%)] Loss:3.606556: , (1.275 sec/step)
0.02315625 accurate

val set:loss5.1409:, (0.924 sec/step)

Train Epoch: 67 iloader:1 [0/213 (0%)] Loss:3.827816: , (1.280 sec/step)
write finish
Train Epoch: 67 il

Train Epoch: 67 iloader:1 [98/213 (46%)] Loss:3.459970: , (1.274 sec/step)
Train Epoch: 67 iloader:1 [99/213 (46%)] Loss:3.335081: , (1.270 sec/step)
Train Epoch: 67 iloader:1 [100/213 (47%)] Loss:3.375494: , (1.274 sec/step)
write finish
Train Epoch: 67 iloader:1 [101/213 (47%)] Loss:3.194259: , (1.272 sec/step)
Train Epoch: 67 iloader:1 [102/213 (48%)] Loss:3.603146: , (1.274 sec/step)
Train Epoch: 67 iloader:1 [103/213 (48%)] Loss:3.275347: , (1.274 sec/step)
Train Epoch: 67 iloader:1 [104/213 (49%)] Loss:3.433964: , (1.272 sec/step)
Train Epoch: 67 iloader:1 [105/213 (49%)] Loss:3.491697: , (1.274 sec/step)
Train Epoch: 67 iloader:1 [106/213 (50%)] Loss:1.335604: , (1.276 sec/step)
Train Epoch: 67 iloader:1 [107/213 (50%)] Loss:3.346256: , (1.277 sec/step)
Train Epoch: 67 iloader:1 [108/213 (51%)] Loss:3.364129: , (1.273 sec/step)
Train Epoch: 67 iloader:1 [109/213 (51%)] Loss:3.569349: , (1.277 sec/step)
Train Epoch: 67 iloader:1 [110/213 (52%)] Loss:2.127969: , (1.277 sec/step)
T